<a href="https://colab.research.google.com/github/MrBillullo/SA-Blueprint/blob/main/SA_Transcript_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install streamlit pyngrok python-docx

In [5]:
%%writefile app.py
import streamlit as st
from datetime import datetime
import ast
import re
from docx import Document
import os

st.set_page_config(page_title="🧠 Solution Engineering Blueprint", layout="wide")
st.title("🧠 Solution Blueprint Intake Assistant")

st.sidebar.header("📥 Paste Pre-Filled Fields (Structured Format)")
st.sidebar.markdown("**Paste GPT output here to auto-fill all fields (Python dict format):**")
pre_filled_dict = st.sidebar.text_area("Paste Python Dictionary Here")

if st.sidebar.button("🧠 Load Fields"):
    try:
        st.session_state.fields = ast.literal_eval(pre_filled_dict)
        st.sidebar.success("Fields loaded successfully!")
    except Exception as e:
        st.sidebar.error(f"⚠️ Invalid dictionary format:\n{e}")
elif "fields" not in st.session_state:
    st.session_state.fields = {}

data = st.session_state.fields

def get(field, default=""):
    return data.get(field, default)

# 🔍 Intake Form
st.header("1. Project Info")
project_name = st.text_input("Project Name", get("project_name"))
project_lead = st.text_input("Project Lead", get("project_lead"))
start_date = st.date_input("Start Date", datetime.strptime(get("start_date", "2025-01-01"), "%Y-%m-%d"))
end_date = st.date_input("End Date", datetime.strptime(get("end_date", "2025-12-31"), "%Y-%m-%d"))

PHASE_1 = ["stakeholders", "objectives", "existing_systems", "current_process", "pain_points", "constraints", "solution_ideas", "quick_vs_long_term", "business_impact", "risks_dependencies"]
PHASE_2 = ["complexity", "analysis_tools", "solution_type", "success_metrics", "business_team_goals", "scalability", "prototyping", "feedback_process", "pros_cons", "workflow_changes", "compliance_standards", "approval_process"]
PHASE_3 = ["assumptions", "connected_teams_systems", "future_growth", "integrations", "technical_challenges", "data_flow", "mvp_testing", "testing_approach", "change_management", "deployment_preference", "performance_tracking", "improvement_process"]
PHASE_4 = ["user_experience", "long_term_support", "additional_stakeholders", "development_methodology", "success_criteria", "automation_opportunities", "security_ethics", "documentation_training"]

st.header("2. Understand the Problem")
responses_phase1 = {field: st.text_area(field.replace("_", " ").title(), get(field)) for field in PHASE_1}
st.header("3. Analyze & Plan")
responses_phase2 = {field: st.text_area(field.replace("_", " ").title(), get(field)) for field in PHASE_2}
st.header("4. Apply & Test")
responses_phase3 = {field: st.text_area(field.replace("_", " ").title(), get(field)) for field in PHASE_3}
st.header("5. Make It Better")
responses_phase4 = {field: st.text_area(field.replace("_", " ").title(), get(field)) for field in PHASE_4}

all_data = {
    "project_name": project_name,
    "project_lead": project_lead,
    "start_date": str(start_date),
    "end_date": str(end_date),
    **responses_phase1,
    **responses_phase2,
    **responses_phase3,
    **responses_phase4
}

def estimate_project_cost(data):
    score = 0
    if "crm" in data.get("integrations", "").lower(): score += 1
    if "dashboard" in data.get("solution_ideas", "").lower(): score += 1
    if "private" in data.get("user_experience", "").lower(): score += 1
    if "analytics" in data.get("success_metrics", "").lower(): score += 1
    if "ai" in data.get("solution_ideas", "").lower() or "chatbot" in data.get("solution_ideas", "").lower(): score += 1

    if score <= 1:
        tier = "Tier 1"
        complexity = "Low"
    elif score <= 3:
        tier = "Tier 2–3"
        complexity = "Medium"
    else:
        tier = "Tier 4–5"
        complexity = "High"

    industry_costs = {
        "Low": (15000, 35000),
        "Medium": (30000, 90000),
        "High": (80000, 150000)
    }

    industry_min, industry_max = industry_costs[complexity]
    adjusted_min = int(industry_min * 0.6)
    adjusted_max = int(industry_max * 0.6)
    hours_min = int(adjusted_min / 120)
    hours_max = int(adjusted_max / 120)

    return {
        "tier": tier,
        "complexity": complexity,
        "cost_range": (adjusted_min, adjusted_max),
        "hours_range": (hours_min, hours_max),
        "rate": "$120/hr"
    }

cost_estimate = estimate_project_cost(all_data)

def generate_docx_report(data, cost_estimate):
    doc = Document()
    doc.add_heading("Solution Engineering Blueprint Report", 0)

    doc.add_paragraph(f"Project Name: {data.get('project_name')}")
    doc.add_paragraph(f"Project Lead: {data.get('project_lead')}")
    doc.add_paragraph(f"Start Date: {data.get('start_date')}")
    doc.add_paragraph(f"End Date: {data.get('end_date')}")
    doc.add_paragraph("")

    def add_section(title, fields):
        doc.add_heading(title, level=1)
        for field in fields:
            label = field.replace("_", " ").title()
            value = data.get(field, "")
            doc.add_paragraph(f"{label}: {value}", style="List Bullet")

    add_section("Phase 1: Understand the Problem", PHASE_1)
    add_section("Phase 2: Analyze & Plan", PHASE_2)
    add_section("Phase 3: Apply & Test", PHASE_3)
    add_section("Phase 4: Make It Better", PHASE_4)

    doc.add_heading("💰 Estimated Development Cost", level=1)
    doc.add_paragraph(f"Tier: {cost_estimate['tier']}")
    doc.add_paragraph(f"Complexity Level: {cost_estimate['complexity']}")
    doc.add_paragraph(f"Estimated Cost Range: ${cost_estimate['cost_range'][0]:,} – ${cost_estimate['cost_range'][1]:,}")
    doc.add_paragraph(f"Estimated Development Time: {cost_estimate['hours_range'][0]} – {cost_estimate['hours_range'][1]} hrs")
    doc.add_paragraph(f"BillulloNex Rate: {cost_estimate['rate']}")

    doc.add_heading("📌 Pricing Methodology", level=2)
    doc.add_paragraph("This estimate is based on industry-standard benchmarks for application development by complexity:")
    doc.add_paragraph("- Low Complexity: $15,000–$35,000")
    doc.add_paragraph("- Medium Complexity: $30,000–$90,000")
    doc.add_paragraph("- High Complexity: $80,000–$150,000")
    doc.add_paragraph("")
    doc.add_paragraph("BillulloNex offers a 40% discount below market value. The estimate excludes additional markups or commissions.")

    filename = f"{re.sub(r'[^a-zA-Z0-9_-]', '_', data.get('project_name', 'project'))}_Blueprint_Report.docx"
    doc.save(filename)
    return filename

# 📥 Export Button
if st.button("📄 Generate Word Report"):
    file_path = generate_docx_report(all_data, cost_estimate)
    with open(file_path, "rb") as f:
        st.download_button(
            label="📥 Download Word Report (.docx)",
            data=f,
            file_name=os.path.basename(file_path),

            mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document"
        )

Overwriting app.py


In [6]:
!ngrok config add-authtoken 2ugBLaE3nBcLJ2pjhGxo7x7Xxos_2n2Ja2ct2BNdBujbKwzp3

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
from pyngrok import ngrok
import threading
import time
import os

# Kill any existing tunnels
ngrok.kill()

# Start Streamlit in background
def run():
    os.system("streamlit run app.py")

thread = threading.Thread(target=run)
thread.start()
time.sleep(5)

# Create ngrok tunnel
public_url = ngrok.connect(8501)
print("🌐 Streamlit app is live at:", public_url)

🌐 Streamlit app is live at: NgrokTunnel: "https://20a4-34-91-15-102.ngrok-free.app" -> "http://localhost:8501"
